### Import Required Libraries and Set Up Environment Variables

In [19]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [20]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
#print(nyt_api_key)
#print(tmdb_api_key)

### Access the New York Times API

In [21]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

print(query_url)
res = requests.get(query_url)
res

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=QZkQel9hvp04mi4RUPJ2HD9vrH0qLoFx&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


<Response [200]>

In [22]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for pages in range(0, 20):

    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{query_url}&page={pages}"

    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    #print(reviews)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {pages}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No articles on page {pages} found")

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [23]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [24]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [25]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [26]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [27]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide

### Access The Movie Database API

In [28]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
#print(tmdb_key_string)


In [29]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles 
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print(f"Sleeping at {request_counter} requests")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)
    data = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = data["results"][0]["id"]

        # Make a request for a the full movie details
        details_query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        # Execute "GET" request with url
        movie_details = requests.get(details_query_url).json()
      
        # Extract the genre names into a list
        genres = []
        for genre in movie_details['genres']:
            genres.append(
                genre["name"]
            )

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in movie_details['spoken_languages']:
            spoken_languages.append(
                language["english_name"]
            )

        # Extract the production_countries' name into a list
        production_countries = []
        for country in movie_details['production_countries']:
            production_countries.append(
                country["name"]
            )

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(
            {
                "title": movie_details['title'],
                "original_title": movie_details['original_title'],
                "budget": movie_details['budget'],
                "genre": genres,
                "language": movie_details['original_language'],
                "spoken_languages": spoken_languages,
                "homepage": movie_details['homepage'],
                "overview": movie_details['overview'],
                "popularity": movie_details['popularity'],
                "runtime": movie_details['runtime'],
                "revenue": movie_details['revenue'],
                "release_date": movie_details['release_date'],
                "vote_average": movie_details['vote_average'],
                "vote_count": movie_details['vote_count'],
                "production_countries": production_countries
            }
        )
    
        # Print out the title that was found
        print(f"'{title}' was found")
    except:
        print(f"'{title}' is not found")



'The Attachment Diaries' was found
'What’s Love Got to Do With It?’ Probably a Lo' is not found
'You Can Live Forever' was found
'A Tourist’s Guide to Love' was found
'Other People’s Children' was found
'One True Loves' was found
'The Lost Weekend: A Love Story' was found
'A Thousand and One' was found
'Your Place or Mine' was found
'Love in the Time of Fentanyl' was found
'The Attachment Diaries' was found
'What’s Love Got to Do With It?’ Probably a Lo' is not found
'You Can Live Forever' was found
'A Tourist’s Guide to Love' was found
'Other People’s Children' was found
'One True Loves' was found
'The Lost Weekend: A Love Story' was found
'A Thousand and One' was found
'Your Place or Mine' was found
'Love in the Time of Fentanyl' was found
'The Attachment Diaries' was found
'What’s Love Got to Do With It?’ Probably a Lo' is not found
'You Can Live Forever' was found
'A Tourist’s Guide to Love' was found
'Other People’s Children' was found
'One True Loves' was found
'The Lost Weekend:

In [30]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(tmdb_movies_list[0:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.181,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]
    },
    

In [31]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.181,102,0,2021-10-07,3.000,4,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.688,96,15055,2023-03-24,6.641,32,"[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",20.280,96,0,2023-04-21,6.329,155,[United States of America]
3,Other People's Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[English, French]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",10.753,104,84178,2022-09-21,6.900,178,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.333,100,37820,2023-04-07,6.455,66,"[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [32]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_reviews_tmdb_df = pd.merge(tmdb_df, reviews_df, how='inner', on='title')
merged_reviews_tmdb_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
4,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [33]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genre", "spoken_languages", "production_countries"]

# Create a list of characters to remove
characters_to_remove = ["[","]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_reviews_tmdb_df[column] = merged_reviews_tmdb_df[column].astype("str")

    # Loop through characters to remove
    for character in characters_to_remove:
        merged_reviews_tmdb_df[column] = merged_reviews_tmdb_df[column].str.replace(character, "",regex=False)

# Display the fixed DataFrame
merged_reviews_tmdb_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
4,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [34]:
# Drop "byline.person" column
drop_column_df = merged_reviews_tmdb_df.drop(columns="byline.person", axis=1)
drop_column_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
2,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
3,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
4,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None


In [35]:
# Delete duplicate rows and reset index
no_duplicates_df = drop_column_df.drop_duplicates().reset_index(drop=True)
no_duplicates_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.181,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.688,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,14.333,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.443,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Portuguese, Spanish",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,12.306,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [36]:
# Export data to CSV without the index
no_duplicates_df.to_csv('nyt_tmdb_data.csv', index=False)